In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("matinsajadi/flights")

print("Path to dataset files:", path)

c:\Users\juand\OneDrive\Escritorio\TripleTen\Sprint 10\Travel-Insurance\travel\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 10.3M/10.3M [00:00<00:00, 19.0MB/s]


Extracting files...
Path to dataset files: C:\Users\juand\.cache\kagglehub\datasets\matinsajadi\flights\versions\1


In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [16]:
df = pd.read_csv (r'C:\Users\juand\OneDrive\Escritorio\TripleTen\Sprint 10\Travel-Insurance\Data\flights.csv')
df = df.drop (['id', 'year', 'dep_time', 'tailnum', 'origin', 'dest', 'carrier', 'flight', 'time_hour', 'sched_arr_time', 'sched_dep_time', 'arr_time'], axis =1)
df = df.dropna (subset=['dep_delay', 'arr_delay'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327346 entries, 0 to 336769
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   month      327346 non-null  int64  
 1   day        327346 non-null  int64  
 2   dep_delay  327346 non-null  float64
 3   arr_delay  327346 non-null  float64
 4   air_time   327346 non-null  float64
 5   distance   327346 non-null  int64  
 6   hour       327346 non-null  int64  
 7   minute     327346 non-null  int64  
 8   name       327346 non-null  object 
dtypes: float64(3), int64(5), object(1)
memory usage: 25.0+ MB


In [22]:
print (df.shape)

(327346, 9)


In [29]:
df_ohe = pd.get_dummies(df, drop_first=True, dtype=int)
X = df_ohe.drop ('arr_delay', axis=1)
y = df_ohe['arr_delay']
x_train, x_valid, y_train,y_valid = train_test_split (X, y, test_size=0.25, random_state=12345)
numeric = ['month', 'day', 'dep_delay', 'air_time', 'distance', 'hour', 'minute']
scaler = StandardScaler ()
scaler.fit (x_train[numeric])
x_train[numeric] = scaler.transform (x_train[numeric])
x_valid[numeric] = scaler.transform (x_valid[numeric])

In [34]:
model = LinearRegression()
model.fit (x_train, y_train)
predicted_valid = model.predict(x_valid)
mse = mean_squared_error(y_valid, predicted_valid)
print("MSE:", mse)
print("RMSE:", np.sqrt(mse))

MSE: 234.72677274680922
RMSE: 15.320795434533066
